In [3]:
import pandas as pd
import networkx as nx

In [9]:
# Загрузка данных
data = pd.read_excel("DynamicData_for_prediction.xlsx")


In [10]:
# Создание графа
G = nx.Graph()

In [11]:
# Добавление вершин (скважин)
wells = ["P1", "P2", "P3", "I1", "I2", "I3"]  # Список скважин
G.add_nodes_from(wells)

In [12]:
# Добавление ребер (связей между скважинами)
# Например, если есть связь между P1 и I1
G.add_edge("P1", "I1")

In [ ]:
# Подготовка данных для обучения модели
# Для каждой скважины создаем временной ряд параметров
well_data = {}
for well in wells:
    parameters = ["WBHP", "WLPR", "WOPR", "WWIR", "WWCT"]
    well_parameters = [f"{parameter}:{well}" for parameter in parameters]
    well_data[well] = data[well_parameters]

In [ ]:
# Пример временного ряда параметров для скважины P1
print(well_data["P1"])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv

In [ ]:
# Определение архитектуры модели
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(5, 16)  # Входной размер 5 (количество параметров), выходной размер 16
        self.conv2 = GCNConv(16, 1)   # Входной размер 16, выходной размер 1

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        return x


In [ ]:
# Преобразование данных в формат PyTorch Geometric
# Необходимо создать объект класса Data для каждой скважины
data_list = []
for well, parameters in well_data.items():
    x = torch.tensor(parameters.values, dtype=torch.float).t()  # Параметры в формате тензора
    edge_index = torch.tensor(G.edges(), dtype=torch.long).t().contiguous()  # Ребра графа
    
    data = Data(x=x, edge_index=edge_index)
    data_list.append(data)

In [ ]:
# Объединяем данные в батчи
loader = DataLoader(data_list, batch_size=1)


In [ ]:
# Инициализация модели и оптимизатора
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Обучение модели
model.train()
for data in loader:
    optimizer.zero_grad()
    out = model(data)
    # Добавьте функцию потерь и обновление параметров модели здесь
